In [ ]:
# Stats Crawler : from Whoscored 
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException

chromedriver = './chromedriver/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

whoscored = 'https://www.whoscored.com/Matches/'
attrs = []
values = []
matchup = []
scored = []


page = np.array([x for x in xrange(58)]) + 335528 # page number

for i in page: 
    try:
        match_report = whoscored + str(i) + '/MatchReport'
        a = driver.get(match_report)
        
        # Check the Season and League Title
        check = driver.find_element_by_xpath("//div[@id='breadcrumb-nav']//a[@href='/Regions/206/Tournaments/4/Seasons/1929']")
        
        # collect the stats of matches
        if check.text == 'La Liga - 2009/2010':
            teams = driver.find_elements_by_class_name('team-link ')
            body = driver.find_element_by_xpath("//div[@id='match-report-team-statistics']")
            score = driver.find_element_by_xpath("//div[@id='match-header']")
       
            for team in teams:
                matchup.append(team.text)
            
            for result in score.find_elements_by_class_name('result'):
                c = result.text.split(":")[0]
                c1 = result.text.split(":")[1]
                scored.append(c)
                scored.append(c1)

            for attr in body.find_elements_by_class_name('stat-label'):
                if i == page[0]:
                    attrs.append("H_" + attr.text)
                    attrs.append("A_" + attr.text)
                else:
                    pass

            for record in body.find_elements_by_class_name('stat-value'):
                values.append(record.text)

        else:
            pass
    except NoSuchElementException: # page isn't exist
        print '# %s' % str(i)
        pass
    
    except WebDriverException:
        print "Stopped by Keyboard or... Something Else."
   

driver.quit()

attrs += ["H_Possesion", "A_Possesion", ]
values = list(zip(*[iter(values)]*len(attrs)))
matchup = list(zip(*[iter(matchup)]*2))
scores = list(zip(*[iter(scored)]*2))
t = pd.DataFrame(data = matchup, columns = ["Home", "Away"])
x = pd.DataFrame(data = values, columns = attrs)
s = pd.DataFrame(data = scores, columns = ["H_Score","A_Score"])
Total = pd.concat([t,x,s], axis=1)

Total.to_csv('la-liga_0910-2.csv')
Total.tail()

In [ ]:
# Rawdata Processing
import pandas as pd
s13 = pd.read_csv("/dataset/bundes1213.csv")

H_Shots = s13.H_Shots
A_Shots = s13.A_Shots
H_Sot = s13.H_Shots_on_target
A_Sot = s13.A_Shots_on_target
H_G = s13.H_Score
A_G = s13.A_Score

STR = pd.concat([H_Sot/(H_Sot + A_Sot), A_Sot/(H_Sot + A_Sot)], axis = 1)
TSR = pd.concat([H_Shots/(H_Shots + A_Shots), A_Sot/(H_Shots + A_Shots)], axis = 1)

H_PDO = ((H_G/H_Sot) + 1 - (A_G/A_Sot)) * 1000
A_PDO = ((A_G/A_Sot) + 1 - (H_G/H_Sot)) * 1000
PDO = pd.concat([H_PDO, A_PDO], axis = 1)

for x in H_Sot[H_Sot == 0].index:
    if A_Sot[x] != 0:
        PDO[0][x] = ((1 - (A_G[x]/float(A_Sot[x]))) * 1000)
        PDO[1][x] = 2000 - PDO[0][x]
    else:
        (PDO[0][x], PDO[1][x]) = (1000, 1000)

for x in A_Sot[A_Sot == 0].index:
    if H_Sot[x] != 0:
        PDO[0][x] = ((1 + (H_G[x]/float(H_Sot[x]))) * 1000)
        PDO[1][x] = 2000 - PDO[0][x]
    else:
        (PDO[0][x], PDO[1][x]) = (1000, 1000)

TSR.columns = ["H_TSR", "A_TSR"]
STR.columns = ["H_STR", "A_STR"]
PDO.columns = ["H_PDO", "A_PDO"]

In [ ]:
data = pd.concat([s13, TSR, STR, PDO], axis = 1)



column = ["Season", "Club_Name", "TSR", "STR", "PDO", "Pass_Success", "Possesion"]

for team in list(set(s13.Home)):
    H = data[data['Home'] == team]
    A = data[data['Away'] == team]
    T_PDO = (sum(H.H_PDO) + sum(A.A_PDO))/34.0
    T_TSR = (sum(H.H_TSR) + sum(A.A_TSR))/34.0
    T_STR = (sum(H.H_STR) + sum(A.A_STR))/34.0
    T_Pass = (sum(H.H_Pass_Success) + sum(A.A_Pass_Success))/38
    T_Poss = (sum(H.H_Possesion) + sum(A.A_Possesion))/38
    print team, T_TSR, T_STR, T_PDO, T_Pass, T_Poss    

In [ ]:
# Crawling the ranked team list in UEFA Coef. Ranking page 
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException #예외 처리 때문에 Import해줌

chromedriver = './chromedriver/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)
year = np.arange(7)+2009
ESP = {}
xpath = "//table[@id='rankingfilters']//thead//tr//th//div[@class='headFilter clearfix']//fieldset[@class='selectBoxSh']//select[@id='clubbycountry']//option[@value='ESP']"

for x in year:
    ESP[x] = []
    ranking = 'http://www.uefa.com/memberassociations/uefarankings/club/season=%s/index.html'% (str(x))
    driver.get(ranking)
    
    button = driver.find_element_by_xpath(xpath)
    button.click()

    clubLnk = driver.find_elements_by_class_name("clubLnk")
    clubOffName = driver.find_elements_by_class_name("clubOffName")
    teams = clubLnk + clubOffName

    for team in teams:
        if team.text != '':
            ESP[x].append(team.text)
            
driver.quit()
            
ESP